In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

In [13]:
#read the csv file
cities_pd = pd.read_csv("../weather/Output/city_weather.csv")
cities_pd

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,57.22,56,40,3.00,AU,1627003747
1,Codrington,-38.2667,141.9667,44.89,90,100,10.25,AU,1627003747
2,Zwedru,6.0667,-8.1281,70.95,98,99,3.00,LR,1627003747
3,Kapaa,22.0752,-159.3190,87.78,82,75,1.99,US,1627003744
4,Diego de Almagro,-26.3667,-70.0500,59.79,34,0,2.42,CL,1627003748
...,...,...,...,...,...,...,...,...,...
557,Luján,-34.5703,-59.1050,59.99,78,100,8.03,AR,1627003804
558,São José da Coroa Grande,-8.8978,-35.1478,73.58,84,28,11.01,BR,1627003877
559,Beltinci,46.6053,16.2406,63.99,70,4,2.75,SI,1627003878
560,Nishihara,26.1842,127.7558,80.65,97,90,25.32,JP,1627003878


In [14]:
gmaps.configure(api_key=g_key)

In [15]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
ideal_df = cities_pd[(cities_pd['Max Temp'] >= 75) & (city_df['Max Temp'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
113,Redmond,47.6740,-122.1215,79.30,42,0,3.00,US,1627003503
146,Ürümqi,43.8010,87.6005,77.95,36,0,4.47,CN,1627003778
168,Aripuanã,-9.1667,-60.6333,75.20,45,0,2.62,BR,1627003782
169,Corinth,37.9407,22.9573,78.69,58,0,3.33,GR,1627003749
206,Suez,29.9737,32.5263,77.16,70,0,5.53,EG,1627003790
213,Poya,-21.3500,165.1500,77.72,56,0,3.53,NC,1627003791
425,Zhezkazgan,47.8043,67.7144,75.90,40,0,9.53,KZ,1627003824
501,Murgab,37.4966,61.9714,76.87,47,0,0.00,TM,1627003865


In [18]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
113,Redmond,47.6740,-122.1215,79.30,42,0,3.00,US,1627003503,
146,Ürümqi,43.8010,87.6005,77.95,36,0,4.47,CN,1627003778,
168,Aripuanã,-9.1667,-60.6333,75.20,45,0,2.62,BR,1627003782,
169,Corinth,37.9407,22.9573,78.69,58,0,3.33,GR,1627003749,
206,Suez,29.9737,32.5263,77.16,70,0,5.53,EG,1627003790,
213,Poya,-21.3500,165.1500,77.72,56,0,3.53,NC,1627003791,
425,Zhezkazgan,47.8043,67.7144,75.90,40,0,9.53,KZ,1627003824,
501,Murgab,37.4966,61.9714,76.87,47,0,0.00,TM,1627003865,


In [19]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
113,Redmond,47.6740,-122.1215,79.30,42,0,3.00,US,1627003503,Archer Hotel Redmond
146,Ürümqi,43.8010,87.6005,77.95,36,0,4.47,CN,1627003778,Sheraton Urumqi Hotel
168,Aripuanã,-9.1667,-60.6333,75.20,45,0,2.62,BR,1627003782,NaN
169,Corinth,37.9407,22.9573,78.69,58,0,3.33,GR,1627003749,Club Hotel Casino Loutraki
206,Suez,29.9737,32.5263,77.16,70,0,5.53,EG,1627003790,Green House
213,Poya,-21.3500,165.1500,77.72,56,0,3.53,NC,1627003791,NaN
425,Zhezkazgan,47.8043,67.7144,75.90,40,0,9.53,KZ,1627003824,Hotel Zumrat
501,Murgab,37.4966,61.9714,76.87,47,0,0.00,TM,1627003865,NaN


In [20]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))